Exercise made by: Happy County

### Exercise 1 - Regex

Link: https://github.com/AndreasVikke/CPH-Business-Python/blob/master/Week9/WeekExcersies/ip-logfile.log

Use regex on the data to get every different ip save the ip’s in a list.

Hint! Data is a CSV file, save regex result to a set to easy deal with duplicates.

In [2]:
import re
import webget

ip_list = []

url = 'https://github.com/AndreasVikke/CPH-Business-Python/blob/master/Week9/WeekExcersies/ip-logfile.log'
webget.download(url)

with open('ip-logfile.log') as file:
    contents = file.read()
    
pattern = re.compile(r'\d+[.]\d+[.]\d+[.]\d+')
result = pattern.findall(contents)

for ip in result:
    ip_list.append(ip)
    ip_set = set(ip_list)
    
print(ip_set)

{'21.15.46.55', '213.179.54.190', '2.33.66.07', '162.247.74.217', '217.170.205.107', '68.184.5.12', '199.249.230.69', '3.95.29.25', '19.2.45.3', '199.249.230.123', '108.87.82.181', '2.12.51.56', '185.107.47.171', '185.220.101.25', '7.59.4.07', '27.02.48.11', '4.96.46.65', '69.31.21.2', '2.2.82.64', '139.99.172.11', '185.220.102.4', '216.151.180.191', '174.130.215.196', '193.31.53.501', '50.35.73.6', '1.23.82.72', '89.163.239.216', '5.199.130.188', '209.141.34.95', '185.220.101.6', '193.218.118.90', '3.42.31.69', '185.220.101.57'}


### Exercise 2 - Selenium

Use selenium(or hint) to paste an ip from the list to: https://www.whois.com/whois/ and get NetName, NetRange, OrgName, Address, City, StateProv, PostalCode Country, RegDate.

Hint! You can use https://whois.whoisxmlapi.com/ instead of selenium

In [7]:
from selenium import webdriver
import random
import pandas as pd

browser = webdriver.Chrome('./chromedriver')
browser.get('https://www.whois.com/whois/')

ip = ip_list[random.randint(0,len(ip_set)-1)]

browser.find_element_by_id("query").send_keys(ip)
browser.find_element_by_tag_name("button").click()

text = browser.find_element_by_tag_name("pre").text

information = {}

information["ip"] = ip

def get_attribute(pattern, name, group):
    compiler = re.compile(pattern)
    search = compiler.search(text)
    if search is None:
        information[name] = None
    else:
        information[name] = search.group(group)

get_attribute(r'(n|N)et(n|N)ame: *(.*)', 'Net-name', 3)
get_attribute(r'(n|N)et(r|R)ange: *(.*)', 'Net-range', 3)
get_attribute(r'(o|O)rg(n|N)ame: *(.*)', 'Org-name', 3)
get_attribute(r'(a|A)ddress: *(.*)', 'Address', 2)
get_attribute(r'City: *(.*)', 'City', 1)
get_attribute(r'StateProv: *(.*)', 'State-prov', 1)
get_attribute(r'PostalCode: *(.*)', 'Postal-code', 1)
get_attribute(r'Country: *(.*)', 'Country', 1)
get_attribute(r'(r|R)eg(d|D)ate: *(.*)', 'Reg-date', 3)
                      
print(information)

{'ip': '68.184.5.12', 'Net-name': 'CLNTN-MO-68-184-0', 'Net-range': '68.184.0.0 - 68.184.15.255', 'Org-name': 'Charter Communications', 'Address': '6399 S Fiddlers Green Circle', 'City': 'Greenwood Village', 'State-prov': 'CO', 'Postal-code': '80111', 'Country': 'US', 'Reg-date': '2003-01-02'}


### Exercise 3 - Data persistence and Web services

Store some of the data in a database with PyMySQL and create a flask server with a GET endpoint to show all the data stored in the DB. (Optionally deploy flask server on your droplet)

In [8]:
import pymysql as sql

cnx = sql.connect(user='dev', password='ax2',host='127.0.0.1',port=3307,db='python')

def persist_data(connector, dictionary, table_name):
    
    cursor = cnx.cursor(sql.cursors.DictCursor)
    #cursor.execute("CREATE TABLE week9 (ip VARCHAR(255), net_name VARCHAR(255), net_range VARCHAR(255), org_name VARCHAR(255), address VARCHAR(255), city VARCHAR(255), state_prov VARCHAR(255), postal_code VARCHAR(255), country VARCHAR(255), reg_date VARCHAR(255))")
    add_info = ("INSERT INTO week9 (ip, net_name, net_range, org_name, address, city, state_prov, postal_code, country, reg_date) VALUES (%(ip)s, %(Net-name)s, %(Net-range)s, %(Org-name)s, %(Address)s, %(City)s, %(State-prov)s, %(Postal-code)s, %(Country)s, %(Reg-date)s)")
    
    cursor.execute(add_info, information)
    cnx.commit()
    cursor.close()
    cnx.close()

persist_data(cnx, information, 'week9')